In [68]:
import numpy as np
import numpy.random as npr
import pandas as pd
import time 

In [69]:
class Plat : 
    def __init__(self ,ide , nom , pmin , pmax , stock):
        self.id = ide 
        self.nom = nom 
        self.pmin = pmin
        self.pmax = pmax 
        self.stock = stock 
        pas = (pmax - pmin) / (k - 1)
        self.liste_prix = [(pmin + i * pas) for i in range(k)]
    
    def afficher(self):
        print("id : " , self.id , "\nnom: " , self.nom , "\n(pmin , pmax) : (", self.pmin , "  , " , self.pmax , ")\nstock : " , self.stock )


In [70]:
#Creation p plats 
Np = 10 
k = 10
list_plats = []
for i in range(Np):
    nom = "plat_" + str(i+1)
    ide = i+1 
    pmin = npr.randint(5 , 11)
    pmax = npr.randint(20 , 31)  
    stock = npr.randint(80 , 121)
    list_plats.append(Plat(ide , nom , pmin , pmax , stock))
for pl in list_plats : 
    pl.afficher()

id :  1 
nom:  plat_1 
(pmin , pmax) : ( 9   ,  27 )
stock :  109
id :  2 
nom:  plat_2 
(pmin , pmax) : ( 9   ,  29 )
stock :  103
id :  3 
nom:  plat_3 
(pmin , pmax) : ( 9   ,  23 )
stock :  107
id :  4 
nom:  plat_4 
(pmin , pmax) : ( 9   ,  27 )
stock :  103
id :  5 
nom:  plat_5 
(pmin , pmax) : ( 9   ,  28 )
stock :  111
id :  6 
nom:  plat_6 
(pmin , pmax) : ( 6   ,  25 )
stock :  82
id :  7 
nom:  plat_7 
(pmin , pmax) : ( 6   ,  27 )
stock :  100
id :  8 
nom:  plat_8 
(pmin , pmax) : ( 8   ,  29 )
stock :  96
id :  9 
nom:  plat_9 
(pmin , pmax) : ( 5   ,  21 )
stock :  114
id :  10 
nom:  plat_10 
(pmin , pmax) : ( 7   ,  29 )
stock :  112


In [71]:
DEB = time.time()
Df = pd.DataFrame(columns = [ "Client" , "Classa" ,"Classf" ,"Mois" ,"Jour" , "Periode" , "Plat" , "Note" , "Prix" , "Achat" ])
Nm = 1 
Nj = 30
Class_j = {"debut":list(range(3 , 10)) , "milieu" : list(range(10 , 25)) , "fin" :[1 , 2] + list(range(25 , 31))}
Class_p = {"debut":list(range(6)) , "milieu" : list(range(6 , 18)) , "fin" :[1 , 2] + list(range(18 , 24))}
nbre_etats = 24 
Class_f  = {"pauvre" : 1/4 , "moyen" : 1/2 , "riche" : 1/4}
Class_a  = {"aime_pas" : 1/4 , "indifferent" : 1/2 , "aime" : 1/4}
Class_j = {"debut":list(range(3 , 10)) , "milieu" : list(range(10 , 25)) , "fin" :[1 , 2] + list(range(25 , Nj+1))}
Class_p = {"debut":list(range(6)) , "milieu" : list(range(6 , 18)) , "fin" :[1 , 2] + list(range(18 , nbre_etats))}
lplnom = []
#Liste des noms des plats 
for plat in list_plats :
    lplnom.append(plat.nom)
Class_j_vals = []
Class = [Class_f , Class_a]
client = 0
for mois in range( 1 , Nm+1):  
    print("mois : " , mois)
    lamda = npr.uniform(20, 30) #Nbre moyen de client par période dans ce mois 
    prix_pauvre_m = npr.uniform(5, 15)
    prix_moyen_m = npr.uniform(1.5, 2) * prix_pauvre_m
    prix_riche_m = npr.uniform(2, 3) * prix_moyen_m
    Class_prix_m = [prix_pauvre_m , prix_moyen_m ,  prix_riche_m]
    for jour in range(1 , Nj+1):
        deb = time.time()
        print("jour : " , jour)
        lamda2 = npr.poisson(lamda) #Nbre moyen de client par période dans la journée en cours 
        Class_prix_j =[]        
        for cpt in range(len(Class_prix_m)):            
            Class_prix_j.append(npr.poisson(Class_prix_m[cpt]))
        #if jour in Class_j["debut"]:
        if (jour>=3 and jour<10):
            Class_prix_j = [npr.uniform(1 , 1.5)*temp for temp in Class_prix_j]
        #if jour in Class_j["fin"]:
        if (jour>=25 or jour <3):
            Class_prix_j = [npr.uniform(1 , 2)*temp for temp in Class_prix_j]
            
        for periode in range(nbre_etats):
            Class_prix_p =[] 
            for cpt in range(len(Class_prix_j)):            
                Class_prix_p.append(npr.poisson(Class_prix_j[cpt]))
            if periode in Class_p["milieu"]:
                Class_prix_p = [npr.uniform(1 , 1.5)*temp for temp in Class_prix_p]
            if periode in Class_p["fin"]:
                Class_prix_p = [npr.uniform(1 , 2)*temp for temp in Class_prix_p]
                
            Dic_prix = {}
            for plat in list_plats : 
                Dic_prix[plat] = npr.choice(plat.liste_prix)
            N = npr.poisson(lamda2) #Nbre de client à la période en cours 
            prop_seg_f = npr.multinomial(N, list(Class_f.values())) #differentes proportion de chaque classe
            for cpt_f in range(len(Class_f)):
                prop_seg_a = npr.multinomial(prop_seg_f[cpt_f] , list(Class_a.values()))
                for cpt_a in range(len(prop_seg_a)):    
                    for pers in range(prop_seg_a[cpt_a]):
                        client+=1 #id client
                        #DataFrame pour chaque client 
                        dp = pd.DataFrame(columns = [ "Mois" , "Jour","Periode","Client" ,"Classf", "Classa" , "Plat","Note" ,"Prix" ,"Propension" , "Achat"]) 
                        Dic_proppay = [] #Liste Propension à  payer pour chaque plat
                        propmax = 0 
                        Note = [] #Liste des notes 
                        Prix = list(Dic_prix.values()) #Liste des prix Choisis 
                        for ind in range(Np):
                            if cpt_a == 0 : #Aime pas 
                                note = npr.randint(1 , 4)
                            if cpt_a ==1 :  #Indifferent 
                                note = npr.randint(1 , 6)
                            if cpt_a ==2 : #Aime
                                note = npr.randint(3 , 6) 
                            Note.append(note)
                            propension = (note/3) *  npr.poisson(Class_prix_p[cpt_f])
                            Dic_proppay.append(propension)
                            if (propmax < (propension / Dic_prix[plat]) ) : #si on achete , alors on achete le produit qui a le plus grand rapport propension a payer / prix 
                                propmax = propension / Dic_prix[plat]                         
                                platmax = plat
                                indmax = ind 
                        Achat = [0 for i in range(Np)]
                        #On achete ssi la propension à payer est plus grande que le prix 
                        if (propmax >= 1): 
                            Achat[indmax] = 1
                       
                        temp = {"Mois" : mois , "Client" : client , "Classf" : list(Class_f.keys())[cpt_f], "Classa" : list(Class_a.keys())[cpt_a] ,"Propension": Dic_proppay ,"Jour":jour , "Periode" : periode ,"Plat" : lplnom ,"Note" : Note , "Prix" : Prix ,"Achat" : Achat }
                        dp =pd.DataFrame(temp)                   
                        Df = pd.concat([Df ,dp])    
        print(time.time() - deb)
print(time.time() - DEB)

mois :  1
jour :  1


/Users/ouattaraismael/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



5.520222187042236
jour :  2
5.108007192611694
jour :  3
8.974326133728027
jour :  4
14.925742149353027
jour :  5
12.959214210510254
jour :  6
19.840277910232544
jour :  7
14.975568056106567
jour :  8
19.250514030456543
jour :  9
29.162898063659668
jour :  10
22.728139877319336
jour :  11
30.275315046310425
jour :  12
26.078201055526733
jour :  13
32.725218057632446
jour :  14
36.6108078956604
jour :  15
33.35139298439026
jour :  16
23.287186861038208
jour :  17
40.031495809555054
jour :  18
36.481492042541504
jour :  19
45.2095890045166
jour :  20
52.399359941482544
jour :  21
32.2326180934906
jour :  22
70.20117807388306
jour :  23
46.721246004104614
jour :  24
34.92591881752014
jour :  25
53.1688449382782
jour :  26
43.47861313819885
jour :  27
72.53221702575684
jour :  28
61.25277900695801
jour :  29
61.22347807884216
jour :  30
72.84227085113525
1058.4887931346893


In [72]:
print(len(Df))
Df.head(50)

191540


,Achat,Classa,Classf,Client,Jour,Mois,Note,Periode,Plat,Prix,Propension
0,0,indifferent,pauvre,1,1,1,2,0,plat_1,27.000000,4.000000
1,0,indifferent,pauvre,1,1,1,4,0,plat_2,24.555556,13.333333
2,0,indifferent,pauvre,1,1,1,2,0,plat_3,19.888889,4.666667
3,0,indifferent,pauvre,1,1,1,5,0,plat_4,21.000000,6.666667
4,0,indifferent,pauvre,1,1,1,4,0,plat_5,17.444444,9.333333
5,0,indifferent,pauvre,1,1,1,5,0,plat_6,8.111111,10.000000
6,0,indifferent,pauvre,1,1,1,3,0,plat_7,15.333333,7.000000
7,0,indifferent,pauvre,1,1,1,4,0,plat_8,17.333333,5.333333
8,1,indifferent,pauvre,1,1,1,5,0,plat_9,13.888889,13.333333
9,0,indifferent,pauvre,1,1,1,2,0,plat_10,9.444444,6.000000


In [73]:
Df.groupby("Client").Achat.count()

Client
1        10
2        10
3        10
4        10
5        10
         ..
19150    10
19151    10
19152    10
19153    10
19154    10
Name: Achat, Length: 19154, dtype: int64

In [74]:
Df[Df.Achat == 1]

,Achat,Classa,Classf,Client,Jour,Mois,Note,Periode,Plat,Prix,Propension
8,1,indifferent,pauvre,1,1,1,5,0,plat_9,13.888889,13.333333
7,1,indifferent,pauvre,2,1,1,4,0,plat_8,17.333333,13.333333
4,1,indifferent,pauvre,3,1,1,5,0,plat_5,17.444444,10.000000
5,1,indifferent,pauvre,4,1,1,5,0,plat_6,8.111111,15.000000
3,1,aime,pauvre,5,1,1,5,0,plat_4,21.000000,16.666667
...,...,...,...,...,...,...,...,...,...,...,...
9,1,indifferent,riche,19150,30,1,4,23,plat_10,29.000000,85.333333
6,1,indifferent,riche,19151,30,1,5,23,plat_7,17.666667,103.333333
4,1,indifferent,riche,19152,30,1,5,23,plat_5,9.000000,76.666667
8,1,aime,riche,19153,30,1,4,23,plat_9,8.555556,88.000000


Interressons nous au plat p 

In [75]:
Dic_Df = {}
for plat in list_plats:
    Dic_Df[plat] = Df[Df["Plat"]==plat.nom]

In [76]:
dff = Dic_Df[list_plats[0]]
dff

,Achat,Classa,Classf,Client,Jour,Mois,Note,Periode,Plat,Prix,Propension
0,0,indifferent,pauvre,1,1,1,2,0,plat_1,27.0,4.000000
0,0,indifferent,pauvre,2,1,1,4,0,plat_1,27.0,6.666667
0,0,indifferent,pauvre,3,1,1,5,0,plat_1,27.0,5.000000
0,0,indifferent,pauvre,4,1,1,1,0,plat_1,27.0,2.000000
0,0,aime,pauvre,5,1,1,3,0,plat_1,27.0,8.000000
...,...,...,...,...,...,...,...,...,...,...,...
0,0,indifferent,riche,19150,30,1,4,23,plat_1,9.0,72.000000
0,0,indifferent,riche,19151,30,1,1,23,plat_1,9.0,13.666667
0,0,indifferent,riche,19152,30,1,1,23,plat_1,9.0,19.000000
0,0,aime,riche,19153,30,1,3,23,plat_1,9.0,48.000000


In [77]:
temp = dff[dff.Achat == 1]
data_final = temp.groupby(["Jour" , "Periode" , "Prix"]).Achat.count()
aaa = pd.DataFrame(data_final)
aaa

Achat
Jour Periode Prix       
1    0       27.0      2
     1       25.0      5
     2       25.0      6
     3       21.0      1
     5       13.0      3
...                  ...
30   19      9.0       2
     20      11.0      5
     21      17.0      2
     22      15.0      5
     23      9.0       2

[617 rows x 1 columns]

In [78]:
aaa = aaa.reset_index() #aaa.add_suffix.reset_index()
aaa

,Jour,Periode,Prix,Achat
0,1,0,27.0,2
1,1,1,25.0,5
2,1,2,25.0,6
3,1,3,21.0,1
4,1,5,13.0,3
...,...,...,...,...
612,30,19,9.0,2
613,30,20,11.0,5
614,30,21,17.0,2
615,30,22,15.0,5


## Regression linéaire

In [79]:
import statsmodels
from statsmodels import regression
model = statsmodels.regression.linear_model.OLS.from_formula('Achat ~ Prix + Periode + Jour', data = aaa)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Achat   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.4360
Date:                Sun, 15 Dec 2019   Prob (F-statistic):              0.727
Time:                        21:46:47   Log-Likelihood:                -1108.0
No. Observations:                 617   AIC:                             2224.
Df Residuals:                     613   BIC:                             2242.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.3513      0.244      9.645      0.000       1.873       2.830
Prix           0.0024      0.010      0.236      0.814      -0.018       0.023
Periode        0.0093      0.008      1.102      0.271      -0.007       0.026
Jour           0.0017      0.007      0.253      0.800      -0.012       0.015
==============================================================================
Omnibus:                      137.838   Durbin-Watson:                   1.993
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              324.650
Skew:                           1.163   Prob(JB):                     3.19e-71
Kurtosis:                       5.686   Cond. No.                         112.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [80]:
y = aaa['Achat']
X = aaa.drop('Achat' , 1)
print(X)
from sklearn.linear_model import LinearRegression as LR
result = LR().fit(X , y)
result.coef_

     Jour  Periode  Prix
0       1        0  27.0
1       1        1  25.0
2       1        2  25.0
3       1        3  21.0
4       1        5  13.0
..    ...      ...   ...
612    30       19   9.0
613    30       20  11.0
614    30       21  17.0
615    30       22  15.0
616    30       23   9.0

[617 rows x 3 columns]


array([0.00171666, 0.0093068 , 0.0024261 ])

In [81]:
"""
Dic_Df_final = {}
#for plat in list_plats :
p = 0
plat = list_plats[p]
Dfp = Dic_Df[plat]
print (type(Dfp["Prix"]))
Dfp["Intercept"]= np.ones_like(Dfp["Prix"])
data = Dfp.drop("Jour" , 1)
cat_vars=['Periode','Note', 'Class']

for var in cat_vars: 
        print(var)
        cat_list= 'var'+'_'+var    
        cat_list = pd.get_dummies(Dfp[var], prefix=var)  
        print(var)
        data1=data.join(cat_list)    
        print(var)
        data=data1    
        
data_vars=data.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]
data_final=data[to_keep]
data_final["Intercept"]= np.ones_like(data_final["Prix"])
Dic_Df_final[plat] = data_final 
"""#Dummies 


'\nDic_Df_final = {}\n#for plat in list_plats :\np = 0\nplat = list_plats[p]\nDfp = Dic_Df[plat]\nprint (type(Dfp["Prix"]))\nDfp["Intercept"]= np.ones_like(Dfp["Prix"])\ndata = Dfp.drop("Jour" , 1)\ncat_vars=[\'Periode\',\'Note\', \'Class\']\n\nfor var in cat_vars: \n        print(var)\n        cat_list= \'var\'+\'_\'+var    \n        cat_list = pd.get_dummies(Dfp[var], prefix=var)  \n        print(var)\n        data1=data.join(cat_list)    \n        print(var)\n        data=data1    \n        \ndata_vars=data.columns.values.tolist()\nto_keep=[i for i in data_vars if i not in cat_vars]\ndata_final=data[to_keep]\ndata_final["Intercept"]= np.ones_like(data_final["Prix"])\nDic_Df_final[plat] = data_final \n'

## Regression Logistique

In [82]:
"""
import statsmodels.api as sm
y=TT['Achete']
X=TT[table]
Rlm_model = sm.Logit(y , X)
result = Rlm_model.fit()
result.summary()
"""

"\nimport statsmodels.api as sm\ny=TT['Achete']\nX=TT[table]\nRlm_model = sm.Logit(y , X)\nresult = Rlm_model.fit()\nresult.summary()\n"